# Análisis y generación de texto mediante procesos estocásticos

*Guillermo Hoyo Bravo y Marcos Martínez Jiménez*

---

## **INTRODUCCIÓN**

El procesamiento de lenguaje natural ha sido históricamente una de las tareas pendientes de los sistemas informáticos hasta el reciente auge de la ciencia de datos, el machine learning y disciplinas afines. 

A día de hoy se utilizan herramientas muy sofisticadas, como redes neuronales profundas, para resolver tareas de traducción, text to spech, etc. Sin embargo, algunas técnicas mucho más sencillas también obtienen muy buenos resultados para problemas concretos. 


### Objetivo:

En esta práctica se explorará la modelización de distintos idiomas mediante procesos estocásticos discretos con el objetivo de:

* Producir un generador aleatorio de nuevas palabras de ese idioma, es decir, palabras con características similares a las de ese idioma.

* Producir un generador aleatorio de frases de ese idioma a partir de palabras reales.

* Clasificar palabras en distintos idiomas en función de sus características. Detectar palabras que han sido prestadas de otro idioma.

* Clasificar frases en distintos idiomas y valorar lo correctas / plausibles que son.

### Procesos estocásticos discretos:

Un proceso estocástico discreto se puede definir como una variable aleatoria $X_t$ con una distribución $P(X_t=x)$ que puede depender de $t\in \mathbb{N}$. A su vez, una trayectoria del proceso corresponde a una secuencia concreta de valores $(X_1,...,X_t)$ que procede de esa distribución.

Una palabra / frase se puede entender como un proceso estocástico discreto donde $X_t$ representa el símbolo presente en la posición $t$ (una letra o una palabra respectivamente). Así, una palabra o frase concreta corresponde a una trayectoria del proceso.

Una forma sencilla de capturar la dependencia frente a $t$ del proceso es aproximar $P(X_t=x)$ como una función únicamente de los $k$ símbolos que aparecen en la trayectoria antes de la posición $t$:

$P(X_t=x | X_{t-1}, ..., X_{t-k})$

donde $k$ corresponde al orden de la aproximación. Bajo esta formulación existe una probabilidad de transición entre todos los posibles $X_{t-1}, ..., X_{t-k}$ a todos los posibles $X_t$.


Además, el proceso estocástico se puede plantear de 2 formas:

1. El símbolo $X_t$ corresponde a las letras de una palabra, de forma que se manejan probabilidades de transición entre letras y las trayectorias modeladas son palabras.

2. El símbolo $X_t$ corresponde a las palabras de una frase, de forma que se manejan probabilidades de transición entre palabras y las trayectorias modeladas son frases.


Finalmente, el modelo resultante se puede aplicar de 2 maneras:

1. Generación de nuevas trayectorias (palabras o frases) de acuerdo a las probabilidades de transición estimadas.

2. Estimación de la verosimilitud de una trayectoria (probabilidad de obtenerla asumiendo que los datos siguen el modelo; plausibilidad de que siga el modelo) de acuerdo a las probabilidades de transición estimadas. 

### Ajuste del modelo y datasets de entrenamiento

El ajuste del modelo descrito anteriormente consiste únicamente en medir dos tipos de frecuencias en uno o varios textos:
1. Frecuencias de transición entre todos los posibles $X_{t-1}, ..., X_{t-k}$ a todos los posibles $X_t$. Probabilidad con la que un grupo de símbolos de tamaño k es sucedido por otro símbolo.

2. Frecuencias iniciales de todos los posibles $X_{t-1}, ..., X_{t-k}$. Probabilidad con la que se encuentra un grupo de símbolos al principio de una trayectoria (las probabilidades de transición no aplican en ese caso porque no hay un grupo previo de símbolos).

En este trabajo se considerarán 2 idiomas, inglés y español, a través de un dataset (comúnmente llamado corpus en lingüistica) de textos para cada uno: 
* Muestra gratuita del ***NOW Corpus Español*** ([en linea]: https://www.corpusdelespanol.org/now/).
* Muestra gratuita del ***Corpus of Contemporary American English*** ([en linea]: https://www.english-corpora.org/coca/).

Ambos corpus incluyen multitud de textos de distintos formatos. Las versiones utilizadas en el trabajo sufrieron un ligero preprocesamiento previo para separar las entradas por líneas y se pueden encontrar en el documento de la entrega (dentro de corpus.zip).

En el caso de la aproximación por palabras se utilizaron versiones reducidas de ambos datasets (dentro de corpus.zip), ya que el número de posibles combinaciones de palabras crece mucho más rápidamente que el de letras. 


## **IMPLEMENTACIÓN**

### 0. Entorno PySpark y Drive

Instalación y carga de PySpark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
!pip install -q pyspark

     |████████████████████████████████| 281.3 MB 42 kB/s 
     |████████████████████████████████| 198 kB 60.4 MB/s 


In [ ]:
import os # libreria de manejo del sistema operativo
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
from pyspark.sql import SparkSession

APP_NAME = "PDGE-tutorialSpark1"
SPARK_URL = "local[*]"
spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()
spark

In [ ]:
sc = spark.sparkContext

Mount del directorio de Drive con los datasets.

Para ejecutar correctamente subir los ficheros adicionales al drive que se use.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Otros módulos:

In [ ]:
import numpy as np
import pandas as pd
import time
import pickle

### 1. Carga de los corpus y preprocesamiento

Los corpus están en formato de texto plano, con las frases separadas por lineas y los distintos textos uno a continuación del otro, ya que este modelo trabaja como mucho con frases y no diferencia entre textos.

In [ ]:
!cp "/content/gdrive/My Drive/now_esp_full.txt" .
esp_corpus = sc.textFile("now_esp_full.txt")
esp_corpus.take(5)

['Gran convocatoria para el concurso docente que se realiza en la Escuela Normal Con una inmensa convocatoria de docentes , convocada desde la 7.30 de este lunes en el salón de actos de la Escuela Normal Mariano Moreno , se realizó la primera jornada de el concurso para titularización de los cargos',
 'El día comenzó con las palabras de bienvenidas de las autoridades , quienes hablaron de cientos de docentes que presentaron sus documentos que deben ser analizados por las autoridades',
 'Los cargos fueron 138 , pero esa suma se incrementó debido a que muchos realizaron cambio de escuelas , abriendo otras oportunidades',
 'Las jornadas continuaran este martes debido , precisamente , a el enorme número de docentes presentes',
 'Estuvieron , la presidenta de el CGE , Graciela Bar , el profesor Héctor de la Fuente , vocal de presidencia , el secretario general de Agmer Fabián Peccín y la directora Departamental de Escuela , María del Carmen Tourfini de Córdoba']

In [ ]:
!cp "/content/gdrive/My Drive/coca_eng_full.txt" .
eng_corpus = sc.textFile("coca_eng_full.txt")
eng_corpus.take(5)

['I think it is safe to say that ours is the only dining room in West Los Angeles on whose table -- an eight-foot-long , two-hundred-pound behemoth on which I have taken my meals for many years -- rest piles of photocopies of articles on suicide , all of which were printed in the Encyclopaedia Britannica over the past 220 years',
 'They represent the convergence of two crucially important strands in my own life',
 'Indeed , as I look at the articles , arrayed before me in fanlike tiers , I get the odd feeling that the old oak ruble on which I have eaten so many thousands of dinners has been set not with its customary china and silver but with my intellectual autobiography',
 'My special relationship with the Britannica is of long standing',
 'It is relevant to report that I was a sickly child and stayed home from school a great deal']

In [ ]:
!cp "/content/gdrive/My Drive/now_esp_words.txt" .
esp_corpus_words = sc.textFile("now_esp_words.txt")
esp_corpus_words.take(5)

['Gran convocatoria para el concurso docente que se realiza en la Escuela Normal Con una inmensa convocatoria de docentes ',
 ', convocada desde la 7.30 de este lunes en el salón de actos de la Escuela Normal Mariano Moreno , se realizó la primera jornada de el concurso para titularización de los cargos ',
 'El día comenzó con las palabras de bienvenidas de las autoridades , quienes hablaron de cientos de docentes que presentaron sus documentos que deben ser analizados por las autoridades ',
 'Los cargos fueron 138 , pero esa suma se incrementó debido a que muchos realizaron cambio de escuelas , abriendo otras oportunidades ',
 'Las jornadas continuaran este martes debido , precisamente , a el enorme número de docentes presentes ']

In [ ]:
!cp "/content/gdrive/My Drive/coca_eng_words.txt" .
eng_corpus_words = sc.textFile("coca_eng_words.txt")
eng_corpus_words.take(5)

['I think it is safe to say that ours is the only dining room in West Los Angeles on whose table -- an eight-foot-long , two-hundred-pound behemoth on which I have taken my meals for many years -- rest piles of photocopies of articles on suicide , all of which were printed in the Encyclopaedia Britannica over the past 220 years ',
 'They represent the convergence of two crucially important strands in my own life ',
 'Indeed , as I look at the articles , arrayed before me in fanlike tiers , I get the odd feeling that the old oak ruble on which I have eaten so many thousands of dinners has been set not with its customary china and silver but with my intellectual autobiography ',
 'My special relationship with the Britannica is of long standing ',
 'It is relevant to report that I was a sickly child and stayed home from school a great deal ']

Preprocesamiento: 
* *Paso a minúsculas*: incluir las mayúsculas puede dar lugar a un resultado más fiel al lenguaje, pero aumenta la complejidad y no aporta información realmente relevante. 

* Eliminación de todos los símbolos que no sean letras: deben ser eliminados a la hora de modelizar palabras y también simplifica la modelización de frases.

* Eliminación de lineas vacías: necesario en varias etapas del proceso para evitar errores en etapas posteriores.

* Eliminación de espacios al principio de las líneas y adición de espacios al final. En la aproximación por letras es necesario porque los espacios marcan el final de las palabras y en la de palabras porque separa la última palabra del newline.

* Adición de newline al final de las lineas en la aproximación por palabras, ya que marcan el final de las frases.

In [ ]:
### Eliminación de símbolos que no son letras y paso a minúsculas
### mediante una regla REGEX sencilla

import re

clean_symb = lambda l: re.sub('[\W_0-9]+', ' ', l.lower())
esp_corpus = esp_corpus.map(clean_symb)
eng_corpus = eng_corpus.map(clean_symb)
esp_corpus_words = esp_corpus_words.map(clean_symb)
eng_corpus_words = eng_corpus_words.map(clean_symb)

In [ ]:
esp_corpus.take(5)

['gran convocatoria para el concurso docente que se realiza en la escuela normal con una inmensa convocatoria de docentes convocada desde la de este lunes en el salón de actos de la escuela normal mariano moreno se realizó la primera jornada de el concurso para titularización de los cargos',
 'el día comenzó con las palabras de bienvenidas de las autoridades quienes hablaron de cientos de docentes que presentaron sus documentos que deben ser analizados por las autoridades',
 'los cargos fueron pero esa suma se incrementó debido a que muchos realizaron cambio de escuelas abriendo otras oportunidades',
 'las jornadas continuaran este martes debido precisamente a el enorme número de docentes presentes',
 'estuvieron la presidenta de el cge graciela bar el profesor héctor de la fuente vocal de presidencia el secretario general de agmer fabián peccín y la directora departamental de escuela maría del carmen tourfini de córdoba']

In [ ]:
esp_corpus_words.take(5)

['gran convocatoria para el concurso docente que se realiza en la escuela normal con una inmensa convocatoria de docentes ',
 ' convocada desde la de este lunes en el salón de actos de la escuela normal mariano moreno se realizó la primera jornada de el concurso para titularización de los cargos ',
 'el día comenzó con las palabras de bienvenidas de las autoridades quienes hablaron de cientos de docentes que presentaron sus documentos que deben ser analizados por las autoridades ',
 'los cargos fueron pero esa suma se incrementó debido a que muchos realizaron cambio de escuelas abriendo otras oportunidades ',
 'las jornadas continuaran este martes debido precisamente a el enorme número de docentes presentes ']

In [ ]:
eng_corpus.take(5)

['i think it is safe to say that ours is the only dining room in west los angeles on whose table an eight foot long two hundred pound behemoth on which i have taken my meals for many years rest piles of photocopies of articles on suicide all of which were printed in the encyclopaedia britannica over the past years',
 'they represent the convergence of two crucially important strands in my own life',
 'indeed as i look at the articles arrayed before me in fanlike tiers i get the odd feeling that the old oak ruble on which i have eaten so many thousands of dinners has been set not with its customary china and silver but with my intellectual autobiography',
 'my special relationship with the britannica is of long standing',
 'it is relevant to report that i was a sickly child and stayed home from school a great deal']

In [ ]:
eng_corpus_words.take(5)

['i think it is safe to say that ours is the only dining room in west los angeles on whose table an eight foot long two hundred pound behemoth on which i have taken my meals for many years rest piles of photocopies of articles on suicide all of which were printed in the encyclopaedia britannica over the past years ',
 'they represent the convergence of two crucially important strands in my own life ',
 'indeed as i look at the articles arrayed before me in fanlike tiers i get the odd feeling that the old oak ruble on which i have eaten so many thousands of dinners has been set not with its customary china and silver but with my intellectual autobiography ',
 'my special relationship with the britannica is of long standing ',
 'it is relevant to report that i was a sickly child and stayed home from school a great deal ']

La eliminación de números produce algunas frases incorrectas pero no son frecuentes asi que no deberían afectar demasiado a los resultados.

In [ ]:
### Eliminación de líneas vacías y espacios

# Filtrado de líneas vacías del input
esp_corpus = esp_corpus.filter(lambda l: len(l)>0)
eng_corpus = eng_corpus.filter(lambda l: len(l)>0)
esp_corpus_words = esp_corpus_words.filter(lambda l: len(l)>0)
eng_corpus_words = eng_corpus_words.filter(lambda l: len(l)>0)

# Eliminación de espacios al principio (si hay un espacio toma el resto de la linea)
esp_corpus = esp_corpus.map(lambda l: l[1:] if l[0]==" " else l)
eng_corpus = eng_corpus.map(lambda l: l[1:] if l[0]==" " else l)
esp_corpus_words = esp_corpus_words.map(lambda l: l[1:] if l[0]==" " else l)
eng_corpus_words = eng_corpus_words.map(lambda l: l[1:] if l[0]==" " else l)

# Filtrado de las líneas vacías generadas (líneas con solo espacio)
esp_corpus = esp_corpus.filter(lambda l: len(l)>0)
eng_corpus = eng_corpus.filter(lambda l: len(l)>0)
esp_corpus_words = esp_corpus_words.filter(lambda l: len(l)>0)
eng_corpus_words = eng_corpus_words.filter(lambda l: len(l)>0)

# Adición de espacio al final si no estaba presente
esp_corpus = esp_corpus.map(lambda l: l+" " if l[-1]!=" " else l)
eng_corpus = eng_corpus.map(lambda l: l+" " if l[-1]!=" " else l)
esp_corpus_words = esp_corpus_words.map(lambda l: l+" " if l[-1]!=" " else l)
eng_corpus_words = eng_corpus_words.map(lambda l: l+" " if l[-1]!=" " else l)

# Filtrado de las líneas que solo tienen un espacio
esp_corpus = esp_corpus.filter(lambda l: l[0]!=" ")
eng_corpus = eng_corpus.filter(lambda l: l[0]!=" ")
esp_corpus_words = esp_corpus_words.filter(lambda l: l[0]!=" ")
eng_corpus_words = eng_corpus_words.filter(lambda l: l[0]!=" ")

# Adición de newline al final si no estaba presente (para palabras)
esp_corpus_words = esp_corpus_words.map(lambda l: l+"\n" if l[-1]!="\n" else l)
eng_corpus_words = eng_corpus_words.map(lambda l: l+"\n" if l[-1]!="\n" else l)

In [ ]:
esp_corpus.take(5)

['gran convocatoria para el concurso docente que se realiza en la escuela normal con una inmensa convocatoria de docentes convocada desde la de este lunes en el salón de actos de la escuela normal mariano moreno se realizó la primera jornada de el concurso para titularización de los cargos ',
 'el día comenzó con las palabras de bienvenidas de las autoridades quienes hablaron de cientos de docentes que presentaron sus documentos que deben ser analizados por las autoridades ',
 'los cargos fueron pero esa suma se incrementó debido a que muchos realizaron cambio de escuelas abriendo otras oportunidades ',
 'las jornadas continuaran este martes debido precisamente a el enorme número de docentes presentes ',
 'estuvieron la presidenta de el cge graciela bar el profesor héctor de la fuente vocal de presidencia el secretario general de agmer fabián peccín y la directora departamental de escuela maría del carmen tourfini de córdoba ']

In [ ]:
esp_corpus_words.take(5)

['gran convocatoria para el concurso docente que se realiza en la escuela normal con una inmensa convocatoria de docentes \n',
 'convocada desde la de este lunes en el salón de actos de la escuela normal mariano moreno se realizó la primera jornada de el concurso para titularización de los cargos \n',
 'el día comenzó con las palabras de bienvenidas de las autoridades quienes hablaron de cientos de docentes que presentaron sus documentos que deben ser analizados por las autoridades \n',
 'los cargos fueron pero esa suma se incrementó debido a que muchos realizaron cambio de escuelas abriendo otras oportunidades \n',
 'las jornadas continuaran este martes debido precisamente a el enorme número de docentes presentes \n']

In [ ]:
eng_corpus.take(5)

['i think it is safe to say that ours is the only dining room in west los angeles on whose table an eight foot long two hundred pound behemoth on which i have taken my meals for many years rest piles of photocopies of articles on suicide all of which were printed in the encyclopaedia britannica over the past years ',
 'they represent the convergence of two crucially important strands in my own life ',
 'indeed as i look at the articles arrayed before me in fanlike tiers i get the odd feeling that the old oak ruble on which i have eaten so many thousands of dinners has been set not with its customary china and silver but with my intellectual autobiography ',
 'my special relationship with the britannica is of long standing ',
 'it is relevant to report that i was a sickly child and stayed home from school a great deal ']

In [ ]:
eng_corpus_words.take(5)

['i think it is safe to say that ours is the only dining room in west los angeles on whose table an eight foot long two hundred pound behemoth on which i have taken my meals for many years rest piles of photocopies of articles on suicide all of which were printed in the encyclopaedia britannica over the past years \n',
 'they represent the convergence of two crucially important strands in my own life \n',
 'indeed as i look at the articles arrayed before me in fanlike tiers i get the odd feeling that the old oak ruble on which i have eaten so many thousands of dinners has been set not with its customary china and silver but with my intellectual autobiography \n',
 'my special relationship with the britannica is of long standing \n',
 'it is relevant to report that i was a sickly child and stayed home from school a great deal \n']

### 2. Cálculo de las aproximaciones por letras de orden $1,...n$

Aproximación de orden 0:

En este caso las frecuencias iniciales son la primera letra de cada palabra y las "frecuencias de transición" son simplemente la frecuencia de cada letra en el texto.


Para calcular las frecuencias iniciales separamos las líneas en palabras con split en base al espacio y un flatmap. A continuación se necesita una etapa de filtrado para eliminar las líneas vacías correspondientes al espacio de final de línea.

In [ ]:
words = esp_corpus.flatMap(lambda l: l.split(" "))
words = words.filter(lambda l: len(l) > 0)
words.take(10)

['gran',
 'convocatoria',
 'para',
 'el',
 'concurso',
 'docente',
 'que',
 'se',
 'realiza',
 'en']

Para calcular la frecuencia se utilizará de forma general la estrategia de convertir el RDD en clave-valor y agregar por clave

In [ ]:
first_lett_freq = words.map(lambda l: (l[0], 1))
first_lett_freq = first_lett_freq.reduceByKey(lambda a,b: a+b)
first_lett_freq.takeOrdered(10, key = lambda a: -a[1])

[('e', 254686),
 ('d', 213676),
 ('l', 175517),
 ('p', 159600),
 ('c', 149573),
 ('a', 144862),
 ('s', 136704),
 ('m', 95159),
 ('q', 81647),
 ('t', 73359)]

También es necesario calcular el total de palabras para normalizar las probabilidades.

In [ ]:
num_words = first_lett_freq.map(lambda l: l[1])
num_words = num_words.reduce(lambda a,b: a+b)
num_words

1961391


El resultado final, la lista de símbolos y sus frecuencias, se guarda temporalmente en formato array. Posteriormente se definirá una clase muy simple para agrupar ambos objetos en el mismo nombre.

In [ ]:
l0 = pd.DataFrame(first_lett_freq.collect())
l0 = l0.sort_values(by=0)
init_key_0, init_prob_0 = np.array(l0[0]), l0[1].to_numpy()/num_words

In [ ]:
init_key_0[:10], init_prob_0[:10]

(array(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j'], dtype=object),
 array([0.07385677, 0.01250898, 0.07625863, 0.10894105, 0.12984968,
        0.01788527, 0.01184822, 0.02421394, 0.02046711, 0.00556442]))

Por otro lado, para calcular las frecuencias de transición separamos las líneas en símbolos y realizamos un proceso de agregación similar al realizado previamente.

In [ ]:
lett_freq = esp_corpus.flatMap(lambda l: list(l))
lett_freq.take(10)

['g', 'r', 'a', 'n', ' ', 'c', 'o', 'n', 'v', 'o']

In [ ]:
lett_freq = lett_freq.map(lambda l: (l,1))
lett_freq = lett_freq.reduceByKey(lambda a,b: a+b)
lett_freq.takeOrdered(10, key = lambda a: -a[1])

[(' ', 1961391),
 ('e', 1254870),
 ('a', 1076909),
 ('o', 795735),
 ('s', 699374),
 ('n', 652376),
 ('r', 599396),
 ('i', 585763),
 ('l', 492608),
 ('d', 454923)]

In [ ]:
num_lett = lett_freq.map(lambda l: l[1])
num_lett = num_lett.reduce(lambda a,b: a+b)
num_lett

11244198

In [ ]:
l0 = pd.DataFrame(lett_freq.collect())
l0 = l0.sort_values(by=0)
trans_key_0, trans_prob_0 = np.array(l0[0]), l0[1].to_numpy()/num_lett

In [ ]:
trans_key_0[:10], trans_prob_0[:10]

(array([' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i'], dtype=object),
 array([0.17443583, 0.09577464, 0.01003086, 0.03711986, 0.04045847,
        0.11160156, 0.00634229, 0.00949859, 0.00657753, 0.05209469]))

Aproximación de orden 1:

En este caso las frecuencias iniciales son equivalentes a las del caso anterior (por lo que se guardan esos resultados en alias correspondientes para $k=1$) pero las frecuencias de transición son las probabilidades de que una letra $a_m$ sea sucedida por una letra $a_n$, por lo que el resultado se deberá almacenar en formato matricial.

A nivel de cálculo se separarán las líneas en pares de símbolos y se agregarán las ocurrencias de $a_m \rightarrow a_n$.

In [ ]:
init_key_1, init_prob_1 = init_key_0, init_prob_0

In [ ]:
pair_freq = esp_corpus.flatMap(lambda l: [l[i:i+2] for i in range(len(l)-1)])
pair_freq.take(10)

['gr', 'ra', 'an', 'n ', ' c', 'co', 'on', 'nv', 'vo', 'oc']

In [ ]:
pair_freq = pair_freq.map(lambda l: (l,1))
pair_freq = pair_freq.reduceByKey(lambda a,b: a+b)
pair_freq.takeOrdered(10, key = lambda a: -a[1])

[('e ', 369767),
 ('a ', 367405),
 ('s ', 327606),
 ('o ', 285970),
 (' e', 238411),
 ('de', 210567),
 (' d', 208959),
 ('en', 206221),
 ('n ', 197647),
 ('es', 191557)]

En este caso la normalización de las frecuencias se hace por filas:

$\sum_n P(a_m\rightarrow a_n) =1$

porque dicha probabilidad es del tipo:
$P(X_t=a_n | X_{t-1}=a_m)$
y al ser una probabilidad condicionada a $a_m$ debe normalizarse por el número de ocurrencias de $a_m$.

La normalización por filas se puede hacer mediante Spark (contando el número de ocurrencias previamente mencionado de forma similar a todos los conteos anteriores), pero es mucho más sencillo hacerlo directamente con numpy ya que las operaciones costosas (el conteo de los grupos de orden $k$) ya están hechas.

In [ ]:
pair_freq = pair_freq.map(lambda l: list(l[0]) + [l[1]])
l1 = pd.DataFrame(pair_freq.collect())
l1 = l1.sort_values(by=[0,1])
l1

,0,1,2
41,,a,137810
677,,b,23470
4,,c,143844
18,,d,208959
638,,e,238411
...,...,...,...
1181,ü,y,1
1133,ü,è,1
1115,ü,é,2
423,ü,í,48


A continuación se convierte el DataFrame de 3 columnas a formato matricial, se rellenan las posiciones vacías con 0s y se recoge en formato numpy.

In [ ]:
m1 = l1.pivot_table(index=0, columns=1, values=2).fillna(0)
trans_keyfrom_1 = np.array(m1.index)
trans_keyto_1 = np.array(m1.columns)
m1 = m1.to_numpy()
m1 = m1/np.sum(m1, axis=1, keepdims=True)
trans_prob_1 = m1

In [ ]:
trans_keyfrom_1[:10], trans_keyto_1[:10]

(array([' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i'], dtype=object),
 array([' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i'], dtype=object))

In [ ]:
trans_prob_1[:5,:5]

array([[0.00000000e+00, 7.35128988e-02, 1.25197572e-02, 7.67316552e-02,
        1.11466380e-01],
       [3.41166245e-01, 4.08576769e-04, 2.73709292e-02, 5.24686858e-02,
        7.38474653e-02],
       [1.81223346e-02, 1.95302733e-01, 1.35651526e-03, 1.79095479e-03,
        1.38311360e-03],
       [6.45929518e-03, 1.62081350e-01, 6.22929060e-05, 1.44064325e-02,
        4.95947367e-04],
       [4.67309852e-02, 1.40652374e-01, 8.57287937e-05, 2.92357168e-04,
        3.95671355e-04]])

Clase para agrupar los array de probabilidad de numpy con los array de índices:

In [ ]:
class ProbObject:

  def __init__(self, prob, keys):
    self.p = np.copy(prob)
    if type(keys) == list:
      self.kf = np.copy(keys[0])
      self.kt = np.copy(keys[1])
    else:
      self.k = np.copy(keys)

In [ ]:
init_1 = ProbObject(init_prob_1, init_key_1)
init_1.p[:10], init_1.k[:10]

(array([0.07385677, 0.01250898, 0.07625863, 0.10894105, 0.12984968,
        0.01788527, 0.01184822, 0.02421394, 0.02046711, 0.00556442]),
 array(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j'], dtype=object))

In [ ]:
trans_1 = ProbObject(trans_prob_1, [trans_keyfrom_1, trans_keyto_1])
trans_1.p[:5,:5], trans_1.kf[:10], trans_1.kt[:10]

(array([[0.00000000e+00, 7.35128988e-02, 1.25197572e-02, 7.67316552e-02,
         1.11466380e-01],
        [3.41166245e-01, 4.08576769e-04, 2.73709292e-02, 5.24686858e-02,
         7.38474653e-02],
        [1.81223346e-02, 1.95302733e-01, 1.35651526e-03, 1.79095479e-03,
         1.38311360e-03],
        [6.45929518e-03, 1.62081350e-01, 6.22929060e-05, 1.44064325e-02,
         4.95947367e-04],
        [4.67309852e-02, 1.40652374e-01, 8.57287937e-05, 2.92357168e-04,
         3.95671355e-04]]),
 array([' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i'], dtype=object),
 array([' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i'], dtype=object))

Función general para calcular las probabilidades de transición de orden $k$:

In [ ]:
def k_approx_lett(k, corpus_rdd, word_rdd):
  '''
  Devuelve la lista de claves y frecuencias de las aproximación de orden k a los
  datos de un rdd
  '''
  ### Probabilidades iniciales

  # Se omiten las palabras demasiado cortas para el orden deseado
  word_rdd = word_rdd.filter(lambda l: len(l) >= k)

  # Se toman el grupo de primeras k letras de cada palabra y se agrega
  first_group_freq = word_rdd.map(lambda l: (l[:k], 1))
  first_group_freq = first_group_freq.reduceByKey(lambda a,b: a+b)

  # Se calcula el número de palabras para normalizar
  num_words = first_group_freq.map(lambda l: l[1])
  num_words = num_words.reduce(lambda a,b: a+b)

  # Paso a dataframe y a formato ProbObject
  l0 = pd.DataFrame(first_group_freq.collect())
  l0 = l0.sort_values(by=0)
  init_key, init_prob = np.array(l0[0]), l0[1].to_numpy()/num_words
  inits = ProbObject(init_prob, init_key)

  ### Probabilidades de transición

  # Separación de las líneas en grupos de símbolos de tamaño k+1
  group_freq = corpus_rdd.flatMap(lambda l: [l[i:i+k+1] for i in range(len(l)-k)])

  # Agregación de acuerdo al grupo de símbolos
  group_freq = group_freq.map(lambda l: (l,1))
  group_freq = group_freq.reduceByKey(lambda a,b: a+b)

  # Paso a dataframe y ProbObject
  group_freq = group_freq.map(lambda l: [l[0][:k], l[0][k], l[1]])
  dataframe = pd.DataFrame(group_freq.collect())
  dataframe = dataframe.sort_values(by=[0,1])
  matrix = dataframe.pivot_table(index=0, columns=1, values=2).fillna(0)
  keysfrom = np.array(matrix.index)
  keysto = np.array(matrix.columns)
  matrix = matrix.to_numpy()
  trans = ProbObject(matrix/np.sum(matrix, axis=1, keepdims=True), [keysfrom, keysto])

  return inits, trans

Cálculos en batch para $k$ en $[1,8]$:

In [ ]:
# El RDD words que se creó en el testeo ya es el del corpus de español
esp_words = words 

esp_stats = {}

start = time.time()
for i in range(1, 8+1):
  print("Calculando aproximación de orden: ", i)
  esp_stats[i] = k_approx_lett(i, esp_corpus, esp_words)
print(f"\nTiempo total de cálculo: {round((time.time() - start) / 60, 3)} minutos")

Calculando aproximación de orden:  1
Calculando aproximación de orden:  2
Calculando aproximación de orden:  3
Calculando aproximación de orden:  4
Calculando aproximación de orden:  5
Calculando aproximación de orden:  6
Calculando aproximación de orden:  7
Calculando aproximación de orden:  8

Tiempo total de cálculo: 4.757 minutos


Probabilidades iniciales para el español con orden 2:

In [ ]:
esp_stats[2][0].k[:10], esp_stats[2][0].p[:10] 

(array(['aa', 'ab', 'ac', 'ad', 'ae', 'af', 'ag', 'ah', 'ai', 'aj'],
       dtype=object),
 array([2.12003992e-05, 2.01295073e-03, 5.71595379e-03, 2.97512269e-03,
        8.53451969e-05, 9.87177564e-04, 1.95859073e-03, 1.28615755e-03,
        3.10395589e-04, 1.83736793e-04]))

Probabilidades de transición para el español con orden 2:

In [ ]:
esp_stats[2][1].kf[:10], esp_stats[2][1].kt[:10], esp_stats[2][1].p[:5,:5] 

(array([' a', ' b', ' c', ' d', ' e', ' f', ' g', ' h', ' i', ' j'],
       dtype=object),
 array([' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i'], dtype=object),
 array([[3.54930702e-01, 2.46716494e-04, 2.61446920e-02, 7.36231043e-02,
         3.53167404e-02],
        [2.75244994e-02, 2.72816361e-01, 2.64167022e-03, 1.40605028e-03,
         1.61908820e-03],
        [4.26156114e-03, 1.79201079e-01, 5.56158060e-05, 2.15511248e-04,
         7.36909430e-04],
        [2.33538637e-03, 2.30475835e-02, 2.87137668e-05, 2.34495762e-04,
         1.29211951e-04],
        [7.62548708e-03, 1.84555243e-04, 9.39553964e-04, 6.86629392e-03,
         9.77723343e-03]]))

In [ ]:
eng_words = eng_corpus.flatMap(lambda l: l.split(" "))

eng_stats = {}

start = time.time()
for i in range(1, 8+1):
  print("Calculando aproximación de orden: ", i)
  eng_stats[i] = k_approx_lett(i, eng_corpus, eng_words)
print(f"\nTiempo total de cálculo: {round((time.time() - start) / 60, 3)} minutos")

Calculando aproximación de orden:  1
Calculando aproximación de orden:  2
Calculando aproximación de orden:  3
Calculando aproximación de orden:  4
Calculando aproximación de orden:  5
Calculando aproximación de orden:  6
Calculando aproximación de orden:  7
Calculando aproximación de orden:  8

Tiempo total de cálculo: 8.746 minutos


Probabilidades iniciales para el inglés con orden 2:

In [ ]:
eng_stats[2][0].k[:10], eng_stats[2][0].p[:10] 

(array(['aa', 'ab', 'ac', 'ad', 'ae', 'af', 'ag', 'ah', 'ai', 'aj'],
       dtype=object),
 array([7.27607868e-05, 3.93538212e-03, 3.90734874e-03, 2.50945969e-03,
        9.07147472e-05, 2.24393007e-03, 2.70947311e-03, 1.84579312e-04,
        8.69979624e-04, 1.70090151e-05]))

Probabilidades de transición para el inglés con orden 2:

In [ ]:
eng_stats[2][1].kf[:10], eng_stats[2][1].kt[:10], eng_stats[2][1].p[:5,:5] 

(array([' a', ' b', ' c', ' d', ' e', ' f', ' g', ' h', ' i', ' j'],
       dtype=object),
 array([' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i'], dtype=object),
 array([[2.01468359e-01, 5.72744118e-04, 3.39378966e-02, 3.30338593e-02,
         2.11325734e-02],
        [7.43818315e-03, 1.30229410e-01, 2.23217500e-04, 2.16016936e-04,
         1.44011290e-05],
        [7.34548983e-03, 1.99274049e-01, 7.30863955e-04, 4.85195399e-04,
         7.67714238e-04],
        [4.04047976e-02, 1.10326087e-01, 2.99850075e-04, 6.65292354e-04,
         3.27961019e-04],
        [1.82104997e-02, 1.00201918e-01, 1.96870268e-03, 2.49369006e-02,
         4.96592630e-02]]))

### 3. Creación de los modelos por letras

Crearemos una clase que actuará como interfaz del modelo, almacenando las probabilidades calculadas y permitiendo la generación de cadenas y cálculo de verosimilitud.

* Inicialización: se guardan los parámetros del modelo y se obtienen las distribuciones de probabilidad cumulativas.

* gen_trayect: genera $n$ trayectorias en base al modelo. Toma muestras uniformes para muestrear las distribuciones de probabilidad del modelo por método de la inversa y considera una trayectoria terminada cuando genera un espacio.

* log_lik: calcula el logaritmo de la verosimilitud de una trayectoria en base a las probabilidades el modelo.

In [ ]:
class lett_estoc_model:

  def __init__(self, k, inits, trans):
    self.k = k
    self.inits = ProbObject(inits.p, inits.k)
    self.trans = ProbObject(trans.p, [trans.kf, trans.kt])

    self.inits.p = np.cumsum(self.inits.p)
    self.trans.p = np.cumsum(self.trans.p, axis=1)


  def gen_trayect(self, n):
    trayects = []
    while len(trayects) != n:
      unif = np.random.uniform()
      trayect = self.inits.k[np.argmax(unif < self.inits.p)]

      last_group = trayect
      sel_letter = "."
      while sel_letter[-1] != " ":
        unif = np.random.uniform()
        last_group_idx = np.where(self.trans.kf == last_group)[0]
        try:
          sel_letter = self.trans.kt[np.argmax(unif < self.trans.p[last_group_idx,:])]
        except:
          break
        trayect = trayect + sel_letter
        last_group = last_group[1:] + sel_letter
      if trayect[-1] == " ":
        trayects.append(trayect)
    return trayects


  def log_lik(self, trayect):
    init_letter_idx = np.where(self.inits.k == trayect[0:self.k])[0]
    if len(init_letter_idx) > 0:
      init_letter_prob = self.inits.p[init_letter_idx] - (self.inits.p[init_letter_idx-1] if init_letter_idx>0 else 0)
    else:
      init_letter_prob = np.array([0])

    if len(init_letter_prob) == 0 or init_letter_prob == 0:
      return np.array([-np.inf])
    else:
      log_lik = np.log(init_letter_prob)

    for i in range(len(trayect)-self.k):
      last_group_idx = self.trans.kf == trayect[i:i+self.k]
      new_letter = np.where(self.trans.kt == trayect[i+self.k])[0]
      if len(new_letter) > 0:
        new_letter_prob = self.trans.p[last_group_idx, new_letter] - (self.trans.p[last_group_idx, new_letter-1] if new_letter>0 else 0)
      else:
        new_letter_prob = np.array([0])

      if len(new_letter_prob) == 0 or new_letter_prob == 0:
        log_lik += -np.inf
      else:
        log_lik += np.log(new_letter_prob)

    return log_lik / len(trayect)
    

Generación de los modelos correspondientes a las probabilidades calculadas previamente:

In [ ]:
esp_models = {}

for i in range(1, 8+1):
  esp_models[i] = lett_estoc_model(i, esp_stats[i][0], esp_stats[i][1])

In [ ]:
eng_models = {}

for i in range(1, 8+1):
  eng_models[i] = lett_estoc_model(i, eng_stats[i][0], eng_stats[i][1])

Los modelos se guardan en archivos para volver a cargarlos una vez han sido generados.

In [ ]:
with open("/content/gdrive/My Drive/esp_models.obj", "wb") as out:
  pickle.dump(esp_models, out)

with open("/content/gdrive/My Drive/eng_models.obj", "wb") as out:
  pickle.dump(eng_models, out)

In [ ]:
with open("/content/gdrive/My Drive/esp_models.obj", "rb") as input:
  esp_models = pickle.load(input)

with open("/content/gdrive/My Drive/eng_models.obj", "rb") as input:
  eng_models = pickle.load(input)

Algunas pruebas con modelos de español:

In [ ]:
# Generación de trayectorias con orden 2
esp_models[2].gen_trayect(10)

['la ',
 'que ',
 'lejanda ',
 'la ',
 'ela ',
 'se ',
 'logen ',
 'es ',
 'ejantenza ',
 'punas ']

In [ ]:
# Generación de trayectorias con orden 6
esp_models[6].gen_trayect(10)

['rentas ',
 'comunes ',
 'sustraer ',
 'barcelona ',
 'atención ',
 'atiborraron ',
 'efectiva ',
 'familiar ',
 'responsables ',
 'soportabilidad ']

In [ ]:
# Cálculo de verosimilitud con orden 2
esp_models[2].log_lik("ilustre ")

array([-2.73273823])

In [ ]:
esp_models[6].log_lik("ilustre ")

array([-1.51984287])

Algunas pruebas con modelos de inglés:

In [ ]:
# Generación de trayectorias con orden 2
eng_models[2].gen_trayect(10)

['mon ',
 'her ',
 'wough ',
 'un ',
 'tee ',
 'the ',
 'of ',
 'whousen ',
 'wan ',
 'se ']

In [ ]:
# Generación de trayectorias con orden 6
eng_models[6].gen_trayect(10)

['supposed ',
 'switch ',
 'having ',
 'sought ',
 'delivery ',
 'cabinet ',
 'struction ',
 'libero ',
 'reference ',
 'association ']

In [ ]:
# Cálculo de verosimilitud con orden 2
eng_models[2].log_lik("manhattan ")

array([-2.41344231])

In [ ]:
eng_models[6].log_lik("manhattan ")

array([-0.96354542])

### 4. Clasificador de palabras entre inglés y español

In [ ]:
class LanguageClasif:

  def __init__(self, models):
    self.models = models

  def predict(self, trayect, k, retmode="predict"):
    log_liks = {}
    for lang in self.models.keys():
      log_liks[lang] = self.models[lang][k].log_lik(trayect)
    if retmode == "predict":
      return max(log_liks, key=log_liks.get)
    elif retmode == "log_liks":
      return log_liks

In [ ]:
esp_eng_clas = LanguageClasif({'esp': esp_models, 'eng': eng_models})

Algunas pruebas con el clasificador de palabras:

In [ ]:
esp_eng_clas.predict("builder ", k=2)

'eng'

In [ ]:
esp_eng_clas.predict("argamasa ", k=2)

'esp'

In [ ]:
esp_eng_clas.predict("builder ", k=3)

'eng'

In [ ]:
esp_eng_clas.predict("villanueva  ", k=4)

'esp'

In [ ]:
esp_eng_clas.predict("builder ", k=2, retmode="log_liks")

{'eng': array([-2.13183133]), 'esp': array([-2.93177783])}

En estas pruebas el clasificador parece capaz de discernir el idioma de las palabras correctamente.

### 5. Cálculo de las aproximaciones por palabras de orden $1,...n$

Las frecuencias iniciales corresponden a la frecuencia del primer grupo de palabras en cada frase y las frecuencias de transición a la frecuencia con que un grupo de palabras es seguida por otra palabra.

Para calcular las frecuencias iniciales recogemos el primer grupo de palabras de cada línea (tomando las k primeras posiciones de split en base al espacio) y agregando. 

Para calcular las frecuencias de transición se separan las líneas en grupos de palabras y se agregan las ocurrencias de $w_{i-k-1},...,w_{i-1} \rightarrow w_{i}$

En este caso el coste computacional es mucho mayor por lo que además de usar un dataset reducido se restringue el $k$ máximo hasta 4. En cualquier caso un $k$ tan alto para la aproximación por palabras da lugar a un overfitting muy alto que solo reproduciría frases literales del corpus.

In [ ]:
def k_approx_word(k, corpus_rdd):
  '''
  Devuelve la lista de claves y frecuencias de las aproximación de orden k a los
  datos de un rdd
  '''
  ### Probabilidades iniciales

  # Se omiten las frases demasiado cortas para el orden deseado
  filt_rdd = corpus_rdd.filter(lambda l: len(l.split(" ")) >= k)

  # Se toman el grupo de primeras k palabras de cada palabra y se agrega
  first_group_freq = filt_rdd.map(lambda l: (tuple(l.split(" ")[:k]), 1))
  first_group_freq = first_group_freq.reduceByKey(lambda a,b: a+b)

  # Se calcula el número de lineas para normalizar
  num_lines = first_group_freq.map(lambda l: l[1])
  num_lines = num_lines.reduce(lambda a,b: a+b)

  # Paso a dataframe y a formato ProbObject
  l0 = pd.DataFrame(first_group_freq.collect())
  l0 = l0.sort_values(by=0)
  init_key, init_prob = np.array(l0[0]), l0[1].to_numpy()/num_lines
  inits = ProbObject(init_prob, init_key)

  ### Probabilidades de transición

  # Separación de las líneas en grupos de palabras de tamaño k+1
  group_freq = filt_rdd.flatMap(lambda l: [tuple(l.split(" ")[i:i+k+1]) for i in range(len(l.split(" "))-k)])

  # Agregación de acuerdo al grupo de símbolos
  group_freq = group_freq.map(lambda l: (l,1))
  group_freq = group_freq.reduceByKey(lambda a,b: a+b)

  # Paso a dataframe y ProbObject
  group_freq = group_freq.map(lambda l: [l[0][:k], l[0][k], l[1]])
  dataframe = pd.DataFrame(group_freq.collect())
  dataframe = dataframe.sort_values(by=[0,1])
  matrix = dataframe.pivot_table(index=0, columns=1, values=2).fillna(0)
  keysfrom = np.array(matrix.index)
  keysto = np.array(matrix.columns)
  matrix = matrix.to_numpy()
  trans = ProbObject(matrix/np.sum(matrix, axis=1, keepdims=True), [keysfrom, keysto])

  return inits, trans

Cálculos en batch para $k$ en $[1,4]$:

In [ ]:
esp_words_stats = {}

start = time.time()
for i in range(1, 4+1):
  print("Calculando aproximación de orden: ", i)
  esp_words_stats[i] = k_approx_word(i, esp_corpus_words)
print(f"\nTiempo total de cálculo: {round((time.time() - start) / 60, 3)} minutos")

Calculando aproximación de orden:  1
Calculando aproximación de orden:  2
Calculando aproximación de orden:  3
Calculando aproximación de orden:  4

Tiempo total de cálculo: 0.307 minutos


Probabilidades iniciales para el español con orden 2:

In [ ]:
esp_words_stats[2][0].k[:5], esp_words_stats[2][0].p[:5] 

(array([('a', 'causa'), ('a', 'el'), ('a', 'fin'), ('a', 'la'),
        ('a', 'los')], dtype=object),
 array([0.00073584, 0.00441501, 0.00073584, 0.00294334, 0.00220751]))

Probabilidades de transición para el español con orden 2:

In [ ]:
esp_words_stats[2][1].kf[:5], esp_words_stats[2][1].kt[:5], esp_words_stats[2][1].p[:5,:5] 

(array([('a', 'adwords'), ('a', 'agresiones'), ('a', 'aguantar'),
        ('a', 'alguien'), ('a', 'alguno')], dtype=object),
 array(['\n', 'a', 'abajo', 'abandonado', 'abandonar'], dtype=object),
 array([[0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.33333333, 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ]]))

Uno de los principales problemas es que al trabajar con palabras hay muchas más combinaciones posibles y con un texto tan reducido muy pocas se ven representadas.

Este es un clásico problema de "maldición de la dimensionalidad".

In [ ]:
eng_words_stats = {}

start = time.time()
for i in range(1, 4+1):
  print("Calculando aproximación de orden: ", i)
  eng_words_stats[i] = k_approx_word(i, eng_corpus_words)
print(f"\nTiempo total de cálculo: {round((time.time() - start) / 60, 3)} minutos")

Calculando aproximación de orden:  1
Calculando aproximación de orden:  2
Calculando aproximación de orden:  3
Calculando aproximación de orden:  4

Tiempo total de cálculo: 0.303 minutos


Probabilidades iniciales para el inglés con orden 2:

In [ ]:
eng_words_stats[2][0].k[:10], eng_words_stats[2][0].p[:10] 

(array([('a', 'an'), ('a', 'bill'), ('a', 'common'),
        ('a', 'comprehensive'), ('a', 'conference'), ('a', 'consequence'),
        ('a', 'dark'), ('a', 'day'), ('a', 'decade'), ('a', 'disheveled')],
       dtype=object),
 array([0.00062422, 0.00062422, 0.00062422, 0.00062422, 0.00062422,
        0.00062422, 0.00062422, 0.00062422, 0.00062422, 0.00062422]))

Probabilidades de transición para el inglés con orden 2:

In [ ]:
eng_words_stats[2][1].kf[:10], eng_words_stats[2][1].kt[:10], eng_words_stats[2][1].p[:5,:5] 

(array([('a', 'an'), ('a', 'babe'), ('a', 'baby'), ('a', 'backbone'),
        ('a', 'balanced'), ('a', 'bamboo'), ('a', 'bar'), ('a', 'bare'),
        ('a', 'basic'), ('a', 'basket')], dtype=object),
 array(['\n', 'a', 'aaron', 'abandoned', 'abilities', 'ability', 'able',
        'ably', 'about', 'above'], dtype=object),
 array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]))

### 6. Creación de los modelos por palabras

Se crea una clase como interfaz del modelo análoga a la utilizada en el caso de las letras:

In [ ]:
class word_estoc_model:

  def __init__(self, k, inits, trans):
    self.k = k
    self.inits = ProbObject(inits.p, inits.k)
    self.trans = ProbObject(trans.p, [trans.kf, trans.kt])

    self.inits.p = np.cumsum(self.inits.p)
    self.trans.p = np.cumsum(self.trans.p, axis=1)


  def gen_trayect(self, n):
    trayects = []
    while len(trayects) != n:
      unif = np.random.uniform()
      trayect = self.inits.k[np.argmax(unif < self.inits.p)]

      last_group = trayect
      sel_word = "."
      while sel_word[-1] != "\n":
        unif = np.random.uniform()
        last_group_idx = np.where([x == last_group for x in self.trans.kf])[0]
        try:
          sel_word = self.trans.kt[np.argmax(unif < self.trans.p[last_group_idx,:])]
        except:
          break
        trayect = trayect + (sel_word, )
        last_group = last_group[1:] + (sel_word, )
      
      if trayect[-1] == "\n":
        trayects.append(" ".join(trayect))
    return trayects


  def log_lik(self, trayect):
    trayect = trayect.split(" ")
    init_letter_idx = np.where(np.array([x == tuple(trayect[0:self.k]) for x in self.inits.k]))[0]
    if len(init_letter_idx) > 0:
      init_letter_prob = self.inits.p[init_letter_idx] - (self.inits.p[init_letter_idx-1] if init_letter_idx>0 else 0)
    else:
      init_letter_prob = np.array([0])

    if len(init_letter_prob) == 0 or init_letter_prob == 0:
      return np.array([-np.inf])
    else:
      log_lik = np.log(init_letter_prob)

    for i in range(len(trayect)-self.k):
      last_group_idx = np.where(np.array([x == tuple(trayect[i:i+self.k]) for x in self.trans.kf]))[0]
      new_word_idx = np.where(self.trans.kt == trayect[i+self.k])[0]
      if len(last_group_idx)>0 and len(new_word_idx)>0:
        new_word_prob = self.trans.p[last_group_idx, new_word_idx] - (self.trans.p[last_group_idx, new_word_idx-1] if new_word_idx>0 else 0)
      else:
        new_word_prob = np.array([0])

      if len(new_word_prob) == 0 or new_word_prob == 0:
        log_lik += -np.inf
      else:
        log_lik += np.log(new_word_prob)

    return log_lik / len(trayect)
    

Generación de los modelos correspondientes a las probabilidades calculadas previamente:

In [ ]:
esp_words_models = {}

for i in range(1, 4+1):
  esp_words_models[i] = word_estoc_model(i, esp_words_stats[i][0], esp_words_stats[i][1])

In [ ]:
eng_words_models = {}

for i in range(1, 4+1):
  eng_words_models[i] = word_estoc_model(i, eng_words_stats[i][0], eng_words_stats[i][1])

Los modelos se guardan en archivos para volver a cargarlos una vez han sido generados.

In [ ]:
with open("/content/gdrive/My Drive/esp_words_models.obj", "wb") as out:
  pickle.dump(esp_words_models, out)

with open("/content/gdrive/My Drive/eng_words_models.obj", "wb") as out:
  pickle.dump(eng_words_models, out)

In [ ]:
with open("/content/gdrive/My Drive/esp_words_models.obj", "rb") as input:
  esp_words_models = pickle.load(input)

with open("/content/gdrive/My Drive/eng_words_models.obj", "rb") as input:
  eng_words_models = pickle.load(input)

Algunas pruebas con modelos de español:

In [ ]:
# Generación de trayectorias con orden 2
esp_words_models[2].gen_trayect(10)

['en sus clases de idioma español \n',
 'los padres a elegir la educación \n',
 'de ahí que uno tarda el resto de la tabla metales y no permitan que sus niveles de profundidad \n',
 'experto en protocolos de telecominicaciones \n',
 'no obstante el premio sí puede considerar se como maestro debemos des estar preparados par cualquier momento en nuestra clase nos habla hacerca de que el radio atómico debería crecer conforme aumenta el tamaño aparente de el instituto de tecnología de california caltech en ee uu \n',
 'quieren ser constantemente el centro de atención directamente sobre su ceo tim cook \n',
 'de los ambientes de trabajo \n',
 'cómo iban a dejar una huella en la educación en casa significa asumir de forma absoluta así que los alumnos contruyan sus propios saberes \n',
 'una vez ha llegado a nuestro público objetivo y por lo que querían se lo damos somos pocos en cambio blogs que no leen apenas hayan leído artículos míos que les encanta la serie pero que implican cambios cons

In [ ]:
# Generación de trayectorias con orden 6
esp_words_models[4].gen_trayect(10)

['este flashback no solo es la respuesta a todo sino que lentamente va jugando con el público dándo le pequeñas pistas que conducirán a una aterradora conclusión \n',
 'y a partir de aquí mi aportación a la discusión \n',
 'indicadora de la capacidad de un elemento para formar cationes \n',
 'sigue leyendo muchos guatemaltecos han crecido o crecieron haciendo el saludo uno colocar se la mano derecha sobre el pecho de el lado de el corazón cuando en los actos cívicos escuchan el himno nacional ya que ante este símbolo patrio y la bandera se deben seguir ciertas normas de comportamiento \n',
 'se debio haber tomado hace mucho tiempo \n',
 'hacemos un recorrido por algunas de ellas por qué interviene ahora francia antigua potencia colonial en los últimos meses francia intentó en reiteradas ocasionespersuadir a los gobiernos de estados unidos y a naciones unidas sobre la necesidad de intervenir en el norte de el país a el quedar fuera de el control estatal se convierta en una zona de creci

In [ ]:
# Cálculo de verosimilitud con orden 2
esp_words_models[2].log_lik("supongo que todos la aplaudían \n")

array([-1.50784765])

In [ ]:
esp_words_models[4].log_lik("se debio haber tomado hace mucho tiempo \n")

array([-0.98415213])

Algunas pruebas con modelos de inglés:

In [ ]:
# Generación de trayectorias con orden 2
eng_words_models[2].gen_trayect(10)

['the wood stove for use after the show so sage comes home with us \n',
 'bering s life when she went to chapter meetings the shonto bureau of indian affairs but not on oilier fish such as the time \n',
 'johnny goldberg i graduated with honors in physics and realized that if i went to the mental hospital but we would have helped the dyalo will come \n',
 'i had no inkling of things to come learn how to best protect the sea and orcas are blocked from entering the lagoon by the time makes a run at the same time increasing numbers of children were being sent to boarding schools were funded a century after i happened upon those suicide notes and toolong \n',
 'now he counted only about two hundred miles to another clinic \n',
 'having assembled a core of committed people kamana and others \n',
 'we headed back to work \n',
 'the show with the next day we get up and follows them \n',
 'it s like this old movie with i think about how to seduce a man in the printings of the general condition

In [ ]:
# Generación de trayectorias con orden 6
eng_words_models[4].gen_trayect(10)

['today my children are grown and my oldest daughter is pregnant with our first grandchild \n',
 'reviving indigenous languages across north america \n',
 'it was a small club \n',
 'several pueblo communities including cochiti acoma and laguna maintain immersion programs that take place in a ceremonial context \n',
 'the walls were thin and i did n t need a lot of talent just an inner \n',
 'soil the month campaign to retake attu and kiska from the japanese \n',
 'we all dropped acid and it s kind of a blur \n',
 'our first stage was to find people \n',
 'as often as i could arrange it \n',
 'or i was doing crystal meth \n']

In [ ]:
# Cálculo de verosimilitud con orden 2
eng_words_models[2].log_lik("paul in the center of albuquerque \n")

array([-2.05275209])

In [ ]:
# Cálculo de verosimilitud con orden 4
eng_words_models[2].log_lik("and he took my talent \n")

array([-1.39330623])

Las trayectorias generadas con $k=2$ corresponden a frases originales (que no están presentes en el corpus). Con $k=4$, sin embargo, prácticamente todas las clases provienen literalmente del corpus.

Por otro lado, con $k=2$ es complicado, pero posible, encontrar una frase con verosimilitud no nula, mientras que con $k=4$ es casi imposible encontrar una frase tal que no provenga del corpus.

Debido a estas limitaciones en la aproximación por palabras no vamos a hacer un clasificador por idioma.

## **RESULTADOS**

### 1. Generadores de palabras aleatorios con distintos órdenes de aproximación

Modelo español, $k=1$:

In [ ]:
esp_models[1].gen_trayect(20)

['hore ',
 'pea ',
 'uestaunirin ',
 'es ',
 'énabonoplle ',
 's ',
 'da ',
 'papren ',
 'lauigiventen ',
 'demí ',
 'y ',
 'san ',
 'codel ',
 'lo ',
 'ca ',
 'esesía ',
 'cimuántiériricr ',
 'pal ',
 'fóneros ',
 'cudapr ']

Modelo español, $k=2$:

In [ ]:
esp_models[2].gen_trayect(20)

['uncios ',
 'ayo ',
 'ya ',
 'andechacción ',
 'denía ',
 'tos ',
 'un ',
 'neada ',
 'de ',
 'larte ',
 're ',
 'amen ',
 'dea ',
 'algure ',
 'ad ',
 'en ',
 'enten ',
 'oblespeureaños ',
 'us ',
 'los ']

Modelo español, $k=3$:

In [ ]:
esp_models[3].gen_trayect(20)

['reducacios ',
 'que ',
 'los ',
 'geligentroecólo ',
 'cales ',
 'mejor ',
 'embariba ',
 'una ',
 'está ',
 'demanetalidar ',
 'hyunos ',
 'recerá ',
 'objeto ',
 'modo ',
 'querio ',
 'parte ',
 'amientar ',
 'incias ',
 'dispectitu ',
 'las ']

Modelo español, $k=4$:

In [ ]:
esp_models[4].gen_trayect(20)

['diagnoticipios ',
 'todos ',
 'unificación ',
 'suelta ',
 'hablamo ',
 'impune ',
 'verdades ',
 'sustodido ',
 'mismo ',
 'hora ',
 'espete ',
 'buena ',
 'fortad ',
 'presentirantiálgicas ',
 'amigos ',
 'producir ',
 'cuando ',
 'formación ',
 'formas ',
 'episodios ']

Modelo español, $k=6$:

In [ ]:
esp_models[6].gen_trayect(20)

['somnolencia ',
 'respaldo ',
 'morenas ',
 'ejecutivas ',
 'social ',
 'infinity ',
 'numerosos ',
 'alemania ',
 'alguna ',
 'wollstonecraft ',
 'percepción ',
 'recogida ',
 'nacional ',
 'descubrí ',
 'ostentarabintantino ',
 'también ',
 'constitución ',
 'juntos ',
 'formalidad ',
 'acerca ']

Modelo español, $k=8$:

In [ ]:
esp_models[8].gen_trayect(20)

['felicitación ',
 'denominada ',
 'artistas ',
 'desarrolla ',
 'jeremías ',
 'convivencia ',
 'eficacia ',
 'privilegio ',
 'palabras ',
 'revolución ',
 'economía ',
 'revistas ',
 'práctica ',
 'pasajeros ',
 'crediticia ',
 'proyectos ',
 'incluido ',
 'formalidad ',
 'naturalizado ',
 'vacilante ']

Podemos comprobar que el modelo es capaz de generar palabras nuevas con características compatibles al español.

Una de las cosas más llamativas, y que es característica de este modelo, es la dependencia del resultado con el $k$ de la aproximación, que actúa como una especie de parámetro de suavizado. 

* Cuánto mayor es $k$ mejor se reproduce la estructura del idioma porque se mantiene más información temporal, pero a la vez se obtienen palabras menos novedosas.

* Por el mismo motivo, un $k$ alto puede dar lugar a sobreajuste frente al corpus concreto utilizado (en vez de reflejar las características del idioma en su totalidad). Un $k$ bajo tiende a evitar el sobreajuste al considerar características simples.

El punto óptimo para la generación de palabras novedosas compatibles con el idioma parece encontrarse en $k\sim [3,4]$.

Modelo inglés, $k=1$:

In [ ]:
eng_models[1].gen_trayect(20)

['micowand ',
 'rs ',
 'fren ',
 'jat ',
 'e ',
 'nome ',
 'ourathe ',
 'paerwe ',
 'ts ',
 'wiorofondea ',
 'ghe ',
 'atinyecocth ',
 'ts ',
 'bonipoher ',
 's ',
 'meistag ',
 'shen ',
 'ppasthe ',
 'h ',
 'e ']

Modelo inglés, $k=2$:

In [ ]:
eng_models[2].gen_trayect(20)

['alin ',
 'handia ',
 'colvaly ',
 'muchs ',
 'rept ',
 'itintaticareve ',
 'emonter ',
 'acteasellpeflarly ',
 'thoreared ',
 'the ',
 'and ',
 'widepleas ',
 'as ',
 'min ',
 'rehintse ',
 'grapposteve ',
 'calgore ',
 'on ',
 'youricalland ',
 'to ']

Modelo inglés, $k=3$:

In [ ]:
eng_models[3].gen_trayect(20)

['next ',
 'nam ',
 'conshing ',
 'aborattrave ',
 'previoles ',
 'ared ',
 'pay ',
 'into ',
 'were ',
 'areath ',
 'fricand ',
 'the ',
 'reven ',
 'list ',
 'and ',
 'cottom ',
 'whit ',
 'her ',
 'fland ',
 'give ']

Modelo inglés, $k=4$:

In [ ]:
eng_models[4].gen_trayect(20)

['closed ',
 'introve ',
 'inmately ',
 'than ',
 'acade ',
 'dire ',
 'mediancertainless ',
 'named ',
 'necess ',
 'trade ',
 'this ',
 'billions ',
 'andred ',
 'different ',
 'literational ',
 'they ',
 'economicided ',
 'still ',
 'barbaris ',
 'would ']

Modelo inglés, $k=6$:

In [ ]:
eng_models[6].gen_trayect(20)

['moynihan ',
 'roaring ',
 'gunkai ',
 'behavioral ',
 'completed ',
 'championship ',
 'should ',
 'patterns ',
 'specifically ',
 'police ',
 'arabia ',
 'tocqueville ',
 'canned ',
 'activism ',
 'street ',
 'personnel ',
 'medicine ',
 'advantage ',
 'awaited ',
 'practice ']

Modelo inglés, $k=8$:

In [ ]:
eng_models[8].gen_trayect(20)

['administration ',
 'investments ',
 'obviously ',
 'children ',
 'memorized ',
 'colombia ',
 'valorized ',
 'christmas ',
 'seasoned ',
 'longitudinal ',
 'politically ',
 'available ',
 'technology ',
 'consistent ',
 'copyright ',
 'including ',
 'desperate ',
 'supermarket ',
 'forefront ',
 'oklahoma ']

El modelo de inglés también es capaz de generar palabras nuevas compatibles con el inglés.

De nuevo se observa el dilema entre novedad y corrección con el valor de $k$ y un punto óptimo para la generación de palabras en $k\sim [3,4]$.

### 2. Verosimilitud de palabras ante los distintos modelos

#### Modelo español:

Primero se comprobará como responde el modelo ante palabras españolas más o menos típicas.

In [ ]:
print("Patata (común): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_models[i].log_lik("patata ")[0])

Patata (común): 
---
k=2: -2.38316301175871
k=3: -2.334836018951301
k=4: -1.9087259381643729


In [ ]:
print("Zapato (común): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_models[i].log_lik("zapato ")[0])

Zapato (común): 
---
k=2: -2.931974834003635
k=3: -2.2047765845494363
k=4: -1.6976607795073753


In [ ]:
print("Conejo (común)")
for i in [2,3,4]:
  print(f"k={i}:", esp_models[i].log_lik("conejo ")[0])

Conejo (común)
k=2: -1.7859342606452249
k=3: -2.402258430309169
k=4: -1.602335930861024


In [ ]:
print("Altramuz (medio): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_models[i].log_lik("altramuz ")[0])

Altramuz (medio): 
---
k=2: -3.3750128868651577
k=3: -inf
k=4: -inf


In [ ]:
print("Recíproco (medio): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_models[i].log_lik("recíproco ")[0])

Recíproco (medio): 
---
k=2: -2.467356082120955
k=3: -2.166416996586181
k=4: -1.9897235721755664


In [ ]:
print("Argamasa (medio): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_models[i].log_lik("argamasa ")[0])

Argamasa (medio): 
---
k=2: -2.521251483931268
k=3: -2.4488315684421442
k=4: -inf


In [ ]:
print("Arrebol (raro): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_models[i].log_lik("arrebol ")[0])

Arrebol (raro): 
---
k=2: -2.8915153029985743
k=3: -2.597948520460691
k=4: -2.704339362521465


In [ ]:
print("Ebúrneo (raro): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_models[i].log_lik("ebúrneo ")[0])

Ebúrneo (raro): 
---
k=2: -inf
k=3: -inf
k=4: -inf


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In [ ]:
print("Jerigonza (raro): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_models[i].log_lik("jerigonza ")[0])

Jerigonza (raro): 
---
k=2: -2.8205811690390923
k=3: -3.064303921628782
k=4: -inf


En estos ejemplos el modelo tiende a asignar mayores verosimilitudes a las palabras que suenan más "normales".

Uno de los motivos de que las palabras "medias" y "raras" tengan baja similitud puede ser la presencia de pares de consonantes, que en general son menos probables que la alternancia de vocales y consonantes.

A continuación se comprueba como responde ante palabras de otros idiomas:

In [ ]:
print("Faculty (inglés): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_models[i].log_lik("faculty ")[0])

Faculty (inglés): 
---
k=2: -2.751128199653689
k=3: -inf
k=4: -inf


In [ ]:
print("Kokoro (japonés): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_models[i].log_lik("kokoro ")[0])

Kokoro (japonés): 
---
k=2: -3.455209641660894
k=3: -inf
k=4: -inf


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In [ ]:
print("Mauvais (francés): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_models[i].log_lik("mauvais ")[0])

Mauvais (francés): 
---
k=2: -4.258738725802563
k=3: -inf
k=4: -inf


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


El modelo tiende a asignar verosimilitudes muy bajas a las palabras de otros idiomas, llegando a considerarlas "imposibles" consistentemente con $k\ge3$.

#### Modelo inglés:

Respuesta del modelo ante palabras inglesas más o menos típicas.

In [ ]:
print("Follow (común): \n---")
for i in [2,3,4]:
  print(f"k={i}:", eng_models[i].log_lik("follow ")[0])

Follow (común): 
---
k=2: -2.115017772053995
k=3: -1.4241036126719224
k=4: -1.1584726063046955


In [ ]:
print("Building (común): \n---")
for i in [2,3,4]:
  print(f"k={i}:", eng_models[i].log_lik("building ")[0])

Building (común): 
---
k=2: -1.9250911656303964
k=3: -1.1046001991798398
k=4: -0.8686828118982648


In [ ]:
print("History (común)")
for i in [2,3,4]:
  print(f"k={i}:", eng_models[i].log_lik("history ")[0])

History (común)
k=2: -1.9929510408858073
k=3: -1.558862375440431
k=4: -1.0380253485076971


In [ ]:
print("Scoundrel (medio): \n---")
for i in [2,3,4]:
  print(f"k={i}:", eng_models[i].log_lik("scoundrel ")[0])

Scoundrel (medio): 
---
k=2: -2.358879409071885
k=3: -2.525295098034643
k=4: -3.073543602555371


In [ ]:
print("Wobble (medio): \n---")
for i in [2,3,4]:
  print(f"k={i}:", eng_models[i].log_lik("wobble ")[0])

Wobble (medio): 
---
k=2: -2.611586622204378
k=3: -2.0419449074183977
k=4: -1.7246108285364283


In [ ]:
print("Gauge (medio): \n---")
for i in [2,3,4]:
  print(f"k={i}:", eng_models[i].log_lik("gauge ")[0])

Gauge (medio): 
---
k=2: -2.825440593538984
k=3: -2.3903229444935272
k=4: -1.7861380852854027


In [ ]:
print("Bamboozled (raro): \n---")
for i in [2,3,4]:
  print(f"k={i}:", eng_models[i].log_lik("bamboozled ")[0])

Bamboozled (raro): 
---
k=2: -2.988277048113295
k=3: -2.880787865257957
k=4: -inf


In [ ]:
print("Flabbergast (raro): \n---")
for i in [2,3,4]:
  print(f"k={i}:", eng_models[i].log_lik("flabbergast ")[0])

Flabbergast (raro): 
---
k=2: -2.6281897790565374
k=3: -2.7438603454998076
k=4: -3.224199924675625


In [ ]:
print("Jentacular (raro): \n---")
for i in [2,3,4]:
  print(f"k={i}:", eng_models[i].log_lik("jentacular ")[0])

Jentacular (raro): 
---
k=2: -2.628788276986164
k=3: -inf
k=4: -inf


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


En estos ejemplos las palabras que suenan "normales" también tienden a tener verosimilitudes más bajas.

A continuación se comprueba como responde ante palabras de otros idiomas:

In [ ]:
print("Especialidad (español): \n---")
for i in [2,3,4]:
  print(f"k={i}:", eng_models[i].log_lik("especialidad ")[0])

Especialidad (español): 
---
k=2: -2.507515242324848
k=3: -1.8289236447340274
k=4: -1.8581488696787585


In [ ]:
print("Kokoro (japonés): \n---")
for i in [2,3,4]:
  print(f"k={i}:", eng_models[i].log_lik("kokoro ")[0])

Kokoro (japonés): 
---
k=2: -4.318474313207795
k=3: -inf
k=4: -inf


In [ ]:
print("Mauvais (francés): \n---")
for i in [2,3,4]:
  print(f"k={i}:", eng_models[i].log_lik("mauvais ")[0])

Mauvais (francés): 
---
k=2: -3.3397266705983526
k=3: -2.885690629208589
k=4: -1.76180877504882


El modelo tiende a asignar verosimilitudes bajas a las palabras de otros idiomas, aunque en este caso 2 tuvieron más verosimilitud qie las palabras "raras" del propio idioma.

### 3. Clasificador de palabras en español o inglés

Inicialmente se probará el clasificador para palabras poco ambiguas.

In [ ]:
print("Builder (inglés): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_eng_clas.predict("builder ", i))

Builder (inglés): 
---
k=2: eng
k=3: eng
k=4: eng


In [ ]:
print("Leisure (inglés): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_eng_clas.predict("leisure ", i))

Leisure (inglés): 
---
k=2: eng
k=3: eng
k=4: eng


In [ ]:
print("Papanatas (español): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_eng_clas.predict("papanatas ", i))

Papanatas (español): 
---
k=2: esp
k=3: esp
k=4: esp


In [ ]:
print("Desierto (español): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_eng_clas.predict("desierto ", i))

Desierto (español): 
---
k=2: esp
k=3: esp
k=4: esp


A continuación se probarán sus resultados con palabras que son de un idioma pero han sido prestadas del otro.

In [ ]:
print("Fútbol (español <- inglés): \n---")
for i in [2,3,4]:
  # Ponemos futbol sin tilde porque el inglés no tiene tilde nunca y siempre da verosimilitud 0
  print(f"k={i}:", esp_eng_clas.predict("futbol ", i, retmode="log_liks"))

Fútbol (español <- inglés): 
---
k=2: {'esp': array([-2.48714509]), 'eng': array([-3.40095363])}
k=3: {'esp': array([-1.6128542]), 'eng': array([-inf])}
k=4: {'esp': array([-1.3876181]), 'eng': array([-inf])}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In [ ]:
print("Vagón (español <- inglés): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_eng_clas.predict("vagon ", i, retmode="log_liks"))

Fútbol (español <- inglés): 
---
k=2: {'esp': array([-2.84950217]), 'eng': array([-2.77276103])}
k=3: {'esp': array([-2.52454593]), 'eng': array([-inf])}
k=4: {'esp': array([-2.40948144]), 'eng': array([-inf])}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In [ ]:
print("Champú (español <- inglés): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_eng_clas.predict("champu ", i, retmode="log_liks"))

Champú (español <- inglés): 
---
k=2: {'esp': array([-3.45973221]), 'eng': array([-2.99436491])}
k=3: {'esp': array([-3.3609506]), 'eng': array([-2.71406035])}
k=4: {'esp': array([-inf]), 'eng': array([-inf])}


In [ ]:
print("Guerilla (inglés <- español): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_eng_clas.predict("guerilla ", i, retmode="log_liks"))

Guerilla (inglés <- español): 
---
k=2: {'esp': array([-2.34552547]), 'eng': array([-2.92188788])}
k=3: {'esp': array([-2.27372412]), 'eng': array([-2.6693376])}
k=4: {'esp': array([-2.22140484]), 'eng': array([-2.42379456])}


In [ ]:
print("Avocado (inglés <- español): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_eng_clas.predict("avocado ", i, retmode="log_liks"))

Avocado (inglés <- español): 
---
k=2: {'esp': array([-2.15325084]), 'eng': array([-2.92026201])}
k=3: {'esp': array([-2.8550493]), 'eng': array([-2.708248])}
k=4: {'esp': array([-2.20789377]), 'eng': array([-2.16127242])}


In [ ]:
print("Vanilla (inglés <- español): \n---")
for i in [2,3,4]:
  print(f"k={i}:", esp_eng_clas.predict("vanilla ", i, retmode="log_liks"))

Vanilla (inglés <- español): 
---
k=2: {'esp': array([-2.34405275]), 'eng': array([-3.06555914])}
k=3: {'esp': array([-2.23719079]), 'eng': array([-2.74545128])}
k=4: {'esp': array([-2.05527703]), 'eng': array([-1.69818001])}


Finalmente hacemos una pequeña prueba con un extracto de los textos del dataset para comprobar si una vez entrenado el modelo es capaz de detectar palabras que posiblemente vienen de otro idioma ($\sim$ detección de outliers).

In [ ]:
esp_text = """sólo los chips de un teléfono inteligente o de una computadora 
              podrían en un futuro próximo arreglarse solos pudiendo detectar 
              fallos en su funcionamiento y recuperarse en microsegundos 
              un equipo de ingenieros de el instituto de tecnología de california
              caltech en eeuu ha desarrollado chips con capacidades autocurativas
              en los experimentos los especialistas utilizaron pequeños 
              amplificadores de baja potencia que contienen un total de setenta 
              y siete chips y destruían varias partes del dispositivo con un 
              láser de alta potencia"""

words = esp_text.split(" ")

eng_pred = []
for word in words:
  if esp_eng_clas.predict(word+" ", 3) == "eng":
    eng_pred.append(word)

print(eng_pred)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


['chips', 'caltech', 'chips', 'chips']


El clasificador ha sido capaz de detectar correctamente la presencia de las palabras "chip" y "caltech" como palabras agenas al idioma.

### 4. Generadores de frases aleatorias con distintos órdenes de aproximación

Modelo español, $k=1$:

In [ ]:
esp_words_models[1].gen_trayect(20)

['una buena meta cognición esta explicar a leer hoy los pasillos de cara como en todo eso hace cuatro elementos que tiene su economía grandes medios es una pareja de la seccional \n',
 'pero ya está de glamour \n',
 'el otro u otras ciudades mas cómoda sin ir \n',
 'hasta los gladiadores es capaz de forma de el agua no por cierto inteligible y enseña que en aquello de otras la falta para hacer lo envenena \n',
 'es la línea de las implicaciones de una manera adecuada ni demasiado elevadas a nivel de youtube \n',
 'ese debate así me hay multitud de el aula para la afinidad electrónica o los metales a largo porque \n',
 'vida personal y si piensan que no les que la popularidad que nos encontramos con el tamaño de la magnitud como el corazón de blogs se vayan para los demás la destrucción ha convertido en la responsabilidad de un problema de una parte de sexto sentido los infantes también más nutritivos no puedo aportar algo también que cometen algunos creen que será tan pocas consecuenci

Modelo español, $k=2$:

In [ ]:
esp_words_models[2].gen_trayect(20)

['las mismas malas condiciones educacionales \n',
 'no son apreciados por el contrario es perjudicial tanto para tener una cantidad de compras \n',
 'como en los laboratorios se pierde la ilusión \n',
 'la excesiva homosexualidad de la administración intranasal de estas tres supertierras han sido descubiertas a el igual que en otros de cine de videojuegos de música \n',
 'las consecuencias de la misma domingo de agosto a las numerosas incógnitas que plantea la operación gala está respaldada a nivel logístico por el mantenimiento de el maestro hacer ver lo contrario es perjudicial tanto para quienes nos leen como para que el avance de las fuerzas islamistas sobre la despenalización de las administraciones públicas pasa también por eliminar parlamentos autonómicos el senado y las tres de vigilancia contra el desinterés de los tres primeros años de trabajar en el colegio viajes a visitar a familiares o compras urgentes \n',
 'dónde está la fórmula y es debido a el cerebro \n',
 'esta mane

Modelo español, $k=4$:

In [ ]:
esp_words_models[4].gen_trayect(20)

['un gran evento de relaciones públicas un perfecto engranaje de propaganda de comunicación gubernamental de sofisticación para seducir a una región enorme potente dinámica que podría ayudar a torcer \n',
 'seguramente queramos potenciar aquellos segmentos en los que rendiremos mejor que en otros por ejemplo a nivel de conversión ventas registros clientes potenciales etc \n',
 'el nuevo modelo de educación a base de competencias es muy buen para lograr el desarrollo de el alumno no solo en el salon de clases sino también para su vida diaria \n',
 'transcurrido un tiempo veremos cómo algunos segmentos que atacamos rinden mejor que otros convierten más convierten mejor son más económicos etc \n',
 'y en este vídeo \n',
 'propiedades periódicas por fin veámoslas \n',
 'que hace mas de años la nueva galería se llama mercar \n',
 'por favor sigue tu ritmo y disfruta como lo haces \n',
 'que hace mas de años la nueva galería se llama mercar \n',
 'en la medida en que le exijas te querrá más 

Modelo inglés, $k=1$:

In [ ]:
eng_words_models[1].gen_trayect(20)

['at dinner mint \n',
 'but changes were not all messy you were the native subsistence hunters \n',
 'she s eight of the draft lottery came forward to drink for alaska s \n',
 'they sought to evolve \n',
 'we boarded a kind of extinction we would not relegate them near my parents and that he d been attracted to escape from the words \n',
 'why teach academic subject would say that i had constituted the rookeries as the wine \n',
 'our reserve in daily lives within the fundamentals of dinners has found rather conducive to be much more beautiful and they had a tanned sea lions and mary s the rolling stone sept \n',
 'their wars with the submission of a bitch i have increased here as somewhat afraid of cold war ended st \n',
 'but will read the theater but it must have been an injunction \n',
 'my own expressed but we have been great many barbiturates \n',
 'once precious skins were beating up and catwalks that a seed in addition a few years old girl drummer \n',
 'photo black roses and j

Modelo inglés, $k=2$:

In [ ]:
eng_words_models[2].gen_trayect(20)

['meantime the prospects for funding the punana leo school said that it can not have a goal namely the conversion of the sterile pack to clamp and cut the pattern for the laboring woman watched \n',
 'as vitus bering s life when she is a major consideration until several years after the birth of the dynamics involved in a decision that changed my life with the st \n',
 'the husband was the founder in of a language and the only thing the walkers arrived in chiang mai just this year and her fresh stories made david s which was perhaps the best jobs in the early sixties and as the time the woman is spiritually joined to both the past years including more than any of cavan s predecessors had shown \n',
 'a measure that might have general pratt turning over in marin while tommy recuperated and we d work on my drumming in the individual and in my generation who speak the language was made a small one lane bridge at its mouth and nose \n',
 'but sea lions in alaska a number of deg hit an inga

Modelo inglés, $k=4$:

In [ ]:
eng_words_models[4].gen_trayect(20)

['the woman took her hair down and removed her jewelry and anything else that might be binding to her \n',
 'here goes my secret spot the \n',
 'between sets he went up handed his card to paine said something \n',
 'you got me \n',
 'photo black white labor in upright supported position \n',
 'have you noticed how people then looked older i mean you look at bogart or those guys from the old movies when they were forty they looked ancient \n',
 'to go for another physical at the draft board i d quit screwing and eat a lot of chocolate for a couple weeks \n',
 'with navajo s validity as a real complex and useful language suddenly nationally acknowledged its usage increased and today this language again is spoken widely \n',
 'the reader can surmise what it meant to me a quarter century after i happened upon those suicide notes to be invited to contribute my own seven pages of printed text headed suicide to volume of the edition of the encyclopaedia britannica was a kind of father figure 

### 5. Evaluación gramática de frases

Debido a las limitaciones computacionales de la aproximación por frases es complicado calcular la verosimilitud de una frase cualquiera (porque es poco probable que sus transiciones aparezcan en el corpus), especialmente con $k$ alto.

En su lugar se harán algunas pruebas con $k=1$ para comprobar si podemos evaluar la corrección gramatical de una frase.

In [ ]:
esp_words_models[1].log_lik("eso ya se sabe")

array([-4.58136529])

In [ ]:
esp_words_models[1].log_lik("ya se sabe eso")

array([-inf])

In [ ]:
esp_words_models[1].log_lik("sabe se ya eso")

array([-inf])

In [ ]:
esp_words_models[1].log_lik("se eso ya sabe")

array([-inf])

El modelo asigna probabilidad nula a versiones alteradas de la frase (incluso a la segunda, que técnicamente es correcta). 

En este caso, este comportamiento probablemente no corresponde a una estimación adecuada de las probabilidades de transición sino a la gran limitación de representatividad del corpus.

In [ ]:
eng_words_models[1].log_lik("i would think of this")

array([-3.54582327])

In [ ]:
eng_words_models[1].log_lik("of this i would think")

array([-inf])

In [ ]:
eng_words_models[1].log_lik("would i think of this")

array([-inf])

In [ ]:
eng_words_models[1].log_lik("think of this i would")

array([-inf])

Lo mismo ocurre con el modelo de inglés.

## **CONCLUSIONES**

* Los modelos del lenguaje en base a procesos estocásticos discretos son una buena herramienta para generación de palabras o frases novedosas, con el orden de aproximación $k$ como regulador del balance entre novedad y adecuación al idioma.

* La aproximación por letras también tiene gran utilidad para evaluar la verosimilitud de una palabra en el idioma, lo que puede aplicarse en investigación lingüística y procesamiento de texto, por ejemplo para:

  1. Cuantificación de la verosimilitud de una palabra a diferentes idiomas modernos e históricos en análisis de etimología.

  2. Detección automática de outliers o palabras prestadas de otros idiomas en un texto

* La aproximación por palabras es mucho más limitada (requiere más recursos computacionales y corpus más grandes), pero igualmente tiene un rendimiento aceptable en la generación de frases.

* Spark es una herramienta muy útil para realizar de forma extremadamente sencilla pero eficiente los cálculos de probabilidad necesarios para ajustar estos modelos.

## **RECONOCIMIENTOS**

Se agradece a Sara Díaz del Ser por su ayuda en la elección del dataset utilizado y sus consejos de cara al interés lingüistico de los modelos estudiados.